In [23]:
import pandas as pd
import argparse
from ftplib import FTP_TLS
import ftplib
import ssl

class SmartFTP(FTP_TLS):
        def makepasv(self):
            invalidhost, port = super(SmartFTP, self).makepasv()
            return self.host, port

        def ntransfercmd(self, cmd, rest=None):
            conn, size = ftplib.FTP.ntransfercmd(self, cmd, rest)
            if self._prot_p:
                conn = self.context.wrap_socket(conn,
                                                server_hostname=self.host,
                                                session=self.sock.session)  # this is the fix
            return conn, size

def download_from_ftp(ftp, filename, date):
    locations = f'/home/elliott/Development/files/penny_pilot_{date}.txt'
    ftp.retrbinary("RETR " + filename, open(locations,"wb").write, 1024)

def get_file(date):
    filelist = []
    ftp = SmartFTP()
    ftp.ssl_version = ssl.PROTOCOL_SSLv23
    ftp.connect('ftp.tradingblock.com', 21, 100)
    ftp.auth()
    ftp.prot_p()
    print(ftp.login(user='tbftpuser02', passwd='Tr@d1ngBfTp85296'))
    ftp.cwd("/penny_pilot/")
    ftp.dir(filelist.append)
    filenames = [line.split()[-1] for line in filelist]
    file = f'penny_pilot_{date}.txt'
    if file in filenames:
        download_from_ftp(ftp, file, date)
        ftp.quit()
        return True  # File was found and downloaded
    else:
        print(f"The file {file} does not exist in the FTP directory.")
        ftp.quit()
        return False  # File was not found

def format_file(date):
    df = pd.read_csv(f'/home/elliott/Development/files/penny_pilot_{date}.txt', delimiter='\t')  # assuming it's comma-separated
    # Filter the DataFrame
    filtered_df = df[df['Type'] != 'Non-Penny']

    # Extract 'Symbol' column and convert it into a list
    penny_list = filtered_df['Symbol'].tolist()
    df_penny = pd.DataFrame(penny_list, columns=['Symbol'])
    df_penny.to_csv("/home/elliott/Development/files/pennies.cfg", index=False)

date = '2023-09-15'
file_downloaded = get_file(date)  # Will return True if file is downloaded, False otherwise

if file_downloaded:
    format_file(date)
else:
    print("Exiting program as the required file was not found.")


230 Login successful.
